# 1. Khai thác mẫu phổ biến và luật kết hợp

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [3]:
spark = SparkSession.builder.appName('lab04').getOrCreate()

22/01/21 15:24:05 WARN Utils: Your hostname, sheepb-HP-Pavilion-Notebook resolves to a loopback address: 127.0.1.1; using 192.168.1.9 instead (on interface wlo1)
22/01/21 15:24:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/21 15:24:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## a. Đọc và xử lí dữ liệu

In [4]:
data_path = './Lab04-Data/'
order_csv = spark.read.csv(data_path + 'orders.csv', inferSchema=True, header=True, sep=',')
product_csv = spark.read.csv(data_path + 'products.csv', inferSchema=True, header=True, sep=',')

In [5]:
order_csv.show()

+--------+----------+-----------------+---------+
|order_id|product_id|add_to_cart_order|reordered|
+--------+----------+-----------------+---------+
|       1|     49302|                1|        1|
|       1|     11109|                2|        1|
|       1|     10246|                3|        0|
|       1|     49683|                4|        0|
|       1|     43633|                5|        1|
|       1|     13176|                6|        0|
|       1|     47209|                7|        0|
|       1|     22035|                8|        1|
|      36|     39612|                1|        0|
|      36|     19660|                2|        1|
|      36|     49235|                3|        0|
|      36|     43086|                4|        1|
|      36|     46620|                5|        1|
|      36|     34497|                6|        1|
|      36|     48679|                7|        1|
|      36|     46979|                8|        1|
|      38|     11913|                1|        0|


In [6]:
product_csv.show()

+----------+--------------------+--------+-------------+
|product_id|        product_name|aisle_id|department_id|
+----------+--------------------+--------+-------------+
|         1|Chocolate Sandwic...|      61|           19|
|         2|    All-Seasons Salt|     104|           13|
|         3|Robust Golden Uns...|      94|            7|
|         4|Smart Ones Classi...|      38|            1|
|         5|Green Chile Anyti...|       5|           13|
|         6|        Dry Nose Oil|      11|           11|
|         7|Pure Coconut Wate...|      98|            7|
|         8|Cut Russet Potato...|     116|            1|
|         9|Light Strawberry ...|     120|           16|
|        10|Sparkling Orange ...|     115|            7|
|        11|   Peach Mango Juice|      31|            7|
|        12|Chocolate Fudge L...|     119|            1|
|        13|   Saline Nasal Mist|      11|           11|
|        14|Fresh Scent Dishw...|      74|           17|
|        15|Overnight Diapers..

In [7]:
merge_df =  order_csv.join(product_csv, 
                             order_csv.product_id == product_csv.product_id)\
                        .groupby('order_id')\
                        .agg(F.collect_list('product_name'))
merge_df.show()

+--------+--------------------------+
|order_id|collect_list(product_name)|
+--------+--------------------------+
|       1|      [Bulgarian Yogurt...|
|     218|      [Natural Artisan ...|
|     473|      [Organic Whole Mi...|
|     631|      [Organic Strawber...|
|     762|      [Organic Strawber...|
|     774|      [Ice Cream Variet...|
|     844|      [Green Beans, Org...|
|     988|      [Natural Vanilla ...|
|    1077|      [Bag of Organic B...|
|    1139|      [Banana, Organic ...|
|    1143|      [Natural Premium ...|
|    1145|      [Banana, Original...|
|    1280|      [Lactose Free Hal...|
|    1342|      [Bag of Organic B...|
|    1350|      [Ground Cinnamon,...|
|    1468|      [Banana, Organic ...|
|    1591|      [Cracked Wheat, O...|
|    1682|      [Bathroom Tissue,...|
|    1721|      [Organic Reduced ...|
|    1890|      [Vanilla Almond B...|
+--------+--------------------------+
only showing top 20 rows



In [8]:
df = merge_df.withColumnRenamed("order_id", "id")\
       .withColumnRenamed("collect_list(product_name)", "products")
df.show()

+----+--------------------+
|  id|            products|
+----+--------------------+
|   1|[Bulgarian Yogurt...|
| 218|[Natural Artisan ...|
| 473|[Organic Whole Mi...|
| 631|[Organic Strawber...|
| 762|[Organic Strawber...|
| 774|[Ice Cream Variet...|
| 844|[Green Beans, Org...|
| 988|[Natural Vanilla ...|
|1077|[Bag of Organic B...|
|1139|[Banana, Organic ...|
|1143|[Natural Premium ...|
|1145|[Banana, Original...|
|1280|[Lactose Free Hal...|
|1342|[Bag of Organic B...|
|1350|[Ground Cinnamon,...|
|1468|[Banana, Organic ...|
|1591|[Cracked Wheat, O...|
|1682|[Bathroom Tissue,...|
|1721|[Organic Reduced ...|
|1890|[Vanilla Almond B...|
+----+--------------------+
only showing top 20 rows



## b. Khai thác tập phổ biến

In [9]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- products: array (nullable = false)
 |    |-- element: string (containsNull = false)



In [16]:
df.select('products').show(1)

+--------------------+
|            products|
+--------------------+
|[Bulgarian Yogurt...|
+--------------------+
only showing top 1 row



In [20]:
!pip install numpy

  Using cached numpy-1.22.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)


In [22]:
from pyspark.ml.fpm import FPGrowth

fpGrowth = FPGrowth(itemsCol="products", minSupport=0.8, minConfidence=0.8)
model = fpGrowth.fit(df)
# Display frequent itemsets.
model.freqItemsets.show()

# Display generated association rules.
model.associationRules.show()

# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(df).show()

+-----+----+
|items|freq|
+-----+----+
+-----+----+

+----------+----------+----------+----+-------+
|antecedent|consequent|confidence|lift|support|
+----------+----------+----------+----+-------+
+----------+----------+----------+----+-------+



+----+--------------------+----------+
|  id|            products|prediction|
+----+--------------------+----------+
|   1|[Bulgarian Yogurt...|        []|
| 218|[Natural Artisan ...|        []|
| 473|[Organic Whole Mi...|        []|
| 631|[Organic Strawber...|        []|
| 762|[Organic Strawber...|        []|
| 774|[Ice Cream Variet...|        []|
| 844|[Green Beans, Org...|        []|
| 988|[Natural Vanilla ...|        []|
|1077|[Bag of Organic B...|        []|
|1139|[Banana, Organic ...|        []|
|1143|[Natural Premium ...|        []|
|1145|[Banana, Original...|        []|
|1280|[Lactose Free Hal...|        []|
|1342|[Bag of Organic B...|        []|
|1350|[Ground Cinnamon,...|        []|
|1468|[Banana, Organic ...|        []|
|1591|[Cracked Wheat, O...|        []|
|1682|[Bathroom Tissue,...|        []|
|1721|[Organic Reduced ...|        []|
|1890|[Vanilla Almond B...|        []|
+----+--------------------+----------+
only showing top 20 rows



# 2. Phân lớp

In [10]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler

In [11]:
df = spark.read.csv(data_path + 'mushrooms.csv', inferSchema=True, header=True, sep=',')
df.show()

+-----+---------+-----------+---------+-------+----+---------------+------------+---------+----------+-----------+----------+------------------------+------------------------+----------------------+----------------------+---------+----------+-----------+---------+-----------------+----------+-------+
|class|cap-shape|cap-surface|cap-color|bruises|odor|gill-attachment|gill-spacing|gill-size|gill-color|stalk-shape|stalk-root|stalk-surface-above-ring|stalk-surface-below-ring|stalk-color-above-ring|stalk-color-below-ring|veil-type|veil-color|ring-number|ring-type|spore-print-color|population|habitat|
+-----+---------+-----------+---------+-------+----+---------------+------------+---------+----------+-----------+----------+------------------------+------------------------+----------------------+----------------------+---------+----------+-----------+---------+-----------------+----------+-------+
|    p|        x|          s|        n|      t|   p|              f|           c|        n|   

## a. Tiền xử lí dữ liệu

In [12]:
features_col = df.columns
label_col = 'class'
features_col.remove(label_col)
featuresIndexer = [StringIndexer(inputCol=column, outputCol=column+"Indexer").fit(df) 
                       for column in features_col]
    
for featureIndexer in featuresIndexer:
    df = featureIndexer.transform(df)
        
features_col = [feature_col + "Indexer" for feature_col in features_col]
vec_assembler = VectorAssembler(inputCols = features_col , outputCol = "features")
features_df = vec_assembler.transform(df).select('features', label_col)

# featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures").fit(features_df)
labelIndexer = StringIndexer(inputCol=label_col, outputCol="indexedLabel").fit(features_df)
# features_df = featureIndexer.transform(features_df)
features_df = labelIndexer.transform(features_df)
    
features_df.show()

22/01/18 00:23:05 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+-----+------------+
|            features|class|indexedLabel|
+--------------------+-----+------------+
|(22,[1,3,4,7,8,9,...|    p|         1.0|
|(22,[1,2,3,4,8,9,...|    e|         0.0|
|(22,[0,1,2,3,4,8,...|    e|         0.0|
|(22,[2,3,4,7,8,9,...|    p|         1.0|
|(22,[1,2,6,8,10,1...|    e|         0.0|
|(22,[2,3,4,8,9,10...|    e|         0.0|
|(22,[0,1,2,3,4,8,...|    e|         0.0|
|(22,[0,2,3,4,8,9,...|    e|         0.0|
|(22,[2,3,4,7,8,9,...|    p|         1.0|
|(22,[0,1,2,3,4,8,...|    e|         0.0|
|(22,[2,3,4,8,9,10...|    e|         0.0|
|(22,[2,3,4,8,9,10...|    e|         0.0|
|(22,[0,1,2,3,4,8,...|    e|         0.0|
|(22,[2,3,4,7,8,9,...|    p|         1.0|
|(22,[1,6,8,10,12,...|    e|         0.0|
|(22,[0,1,2,7,8,9,...|    e|         0.0|
|(22,[0,1,2,6,8,10...|    e|         0.0|
|(22,[1,3,4,7,8,9,...|    p|         1.0|
|(22,[2,3,4,7,8,9,...|    p|         1.0|
|(22,[1,3,4,7,8,9,...|    p|         1.0|
+--------------------+-----+------

In [13]:
# Chia tập dữ liệu ra thành train, test theo tỉ lệ 80:20
train, test = features_df.randomSplit([0.8, 0.2])

## b. Mô hình decision Tree

In [14]:
decision_tree = DecisionTreeClassifier(labelCol='indexedLabel', featuresCol='features', maxDepth=20, maxBins=32)
model_dct = decision_tree.fit(train)

## c. Mô hình random forest

In [15]:
rdf_cls = RandomForestClassifier(labelCol='indexedLabel', featuresCol='features', maxDepth=20, numTrees=10)
model_rdf = rdf_cls.fit(train)

## d. Đánh giá 2 mô hình trên tập kiểm thử

In [16]:
test_pred_ds = model_dct.transform(test)
test_pred_rdf = model_rdf.transform(test)
# accuracy
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", 
    predictionCol="prediction", 
    metricName="accuracy")

accuracy_ds = evaluator.evaluate(test_pred_ds)
accuracy_rdf = evaluator.evaluate(test_pred_rdf)

print("Accuracy on test")
print(f"- accuracy of decisionTree: {accuracy_ds}")
print(f"- accuracy of RandomForest: {accuracy_rdf}") 

Accuracy on test
- accuracy of decisionTree: 1.0
- accuracy of RandomForest: 1.0


## e. Sử dụng Pipeline để thiết lập các bước trên thành một bước duy nhất

In [17]:
from pyspark.ml import Pipeline
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.tuning import TrainValidationSplit

In [30]:
df = spark.read.csv(data_path + 'mushrooms.csv', inferSchema=True, header=True, sep=',')

# processing
features_col = df.columns
features_col.remove(label_col)
featuresIndexer = [StringIndexer(inputCol=column, outputCol=column+"Indexer").fit(df) 
                       for column in features_col]
    
for featureIndexer in featuresIndexer:
    df = featureIndexer.transform(df)
        
features_col = [feature_col + "Indexer" for feature_col in features_col]
vec_assembler = VectorAssembler(inputCols = features_col , outputCol = "features")
features_df = vec_assembler.transform(df).select('features', label_col)
    

    
train_df, test_df = features_df.randomSplit([0.8, 0.2])

labelIndexer = StringIndexer(inputCol=label_col, outputCol="indexedLabel").fit(features_df)

decision_tree = DecisionTreeClassifier(labelCol='indexedLabel', featuresCol='features', maxDepth=20, maxBins=32)

pipeline_dct = Pipeline(stages=[labelIndexer, decision_tree])

params_dct = ParamGridBuilder().addGrid(decision_tree.maxDepth, [5, 10, 20])\
                            .addGrid(decision_tree.maxBins, [ 15, 32])\
                            .build()

evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", 
    predictionCol="prediction", 
    metricName="accuracy")

tvs_dct = TrainValidationSplit().setTrainRatio(0.8)\
                            .setEstimatorParamMaps(params_dct)\
                            .setEstimator(pipeline_dct)\
                            .setEvaluator(evaluator)

tvsFitted = tvs_dct.fit(train_df)
evaluator.evaluate(tvsFitted.transform(test_df))

1.0

In [32]:
rdf_cls = RandomForestClassifier(labelCol='indexedLabel', featuresCol='features', maxDepth=20, numTrees=10)

pipeline_rdf = Pipeline(stages=[labelIndexer, rdf_cls])

params_rdf = ParamGridBuilder().addGrid(rdf_cls.maxDepth, [5, 10, 20])\
                            .addGrid(rdf_cls.numTrees, [ 5, 10, 15])\
                            .build()

tvs_dct = TrainValidationSplit().setTrainRatio(0.8)\
                            .setEstimatorParamMaps(params_rdf)\
                            .setEstimator(pipeline_rdf)\
                            .setEvaluator(evaluator)

tvsFitted = tvs_dct.fit(train_df)
evaluator.evaluate(tvsFitted.transform(test_df))

0.9975713418336369

# 3. GOM CỤM